<a href="https://colab.research.google.com/github/manaribraheem/Bert_Model/blob/main/BERT_Model_14_6_2022_By_Manar_and_Anj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT MODEL

In [ ]:
!pip3 install -q transformers 

In [ ]:
!transformers -version

/bin/bash: transformers -version: command not found


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import torch
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import DistilBertTokenizerFast, AutoTokenizer
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


##### 1. Data

In [ ]:
# custom dataset
liberta_leasing_dataset = pd.read_excel("/content/drive/MyDrive/Bert_classification/Bert_data.xlsx")

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
liberta_leasing_dataset

,BANK_ID,DESCRIPTION,PREDICTION
0,Luxembourg-HSBC-bank-proof-of-address-statemen...,MR SAMPLE RENT,UNKNOWN
1,Hungary-MKB-Bank-statement-template-in-Word-fo...,ATM Widndrawal,CASH
2,Indonesia-Bank-Negara-Indonesia-BNI-proof-of-a...,ATM Withdrawal,CASH
3,Indonesia-Bank-Negara-Indonesia-BNI-proof-of-a...,ATM Withdrawal,CASH
4,Indonesia-Bank-Negara-Indonesia-BNI-proof-of-a...,ATM Withdrawal,CASH
...,...,...,...
2027,First_City_Monument_Bank,"web:App: Loan Repayment, To First Ban",LOAN
2028,First_City_Monument_Bank,KUNSMANN NIG. LTD.,TRANSFER
2029,First_City_Monument_Bank,web:App: To ACCESS BANK PLC (DIAMOND),TRANSFER
2030,First_City_Monument_Bank,web:App: To GTBank Plc ANOZIE IKECHUK,TRANSFER


In [ ]:
liberta_leasing_dataset['PREDICTION'].value_counts()

CHARGES     552
TRANSFER    480
PURCHASE    423
CASH        184
SALARY      108
PAYMENT     102
LOAN         75
REVERSAL     58
UNKNOWN      39
CHEQUE       11
Name: PREDICTION, dtype: int64

In [ ]:
liberta_leasing_dataset.isnull().sum()

BANK_ID        0
DESCRIPTION    0
PREDICTION     0
dtype: int64

In [ ]:
#  list all present columns 
liberta_leasing_dataset.columns

Index(['BANK_ID', 'DESCRIPTION', 'PREDICTION'], dtype='object')

In [ ]:
# limit the dataset to columns of interest (text + label)
ll_dataset = liberta_leasing_dataset[["DESCRIPTION", "PREDICTION"]].astype('str').reset_index(drop=True)

In [ ]:
# You must map the classes (text) to one hot encoded
ll_dataset["PREDICTION"] = ll_dataset["PREDICTION"].map({'TRANSFER':0, 
                              'PURCHASE':1, 
                              'LOAN':2, 
                              'CHARGES':3, 
                              'SALARY':4, 
                              'CASH':5,
                              'REVERSAL':6,
                              'CHEQUE':7,
                              'PAYMENT':8,
                              'UNKNOWN':9})


In [ ]:
ll_dataset['PREDICTION'].value_counts()

3    552
0    480
1    423
5    184
4    108
8    102
2     75
6     58
9     39
7     11
Name: PREDICTION, dtype: int64

###### 2. Augmentation

In [ ]:
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 410 kB 8.3 MB/s 


In [ ]:
import nlpaug.augmenter.word.context_word_embs as aug

In [ ]:
from sklearn.utils import shuffle

In [ ]:
augmenter = aug.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

In [ ]:
def augmentMyData2(df, augmenter, repetitions=1, samples=200):
    augmented_texts = []
    # select only the minority class samples
    spam_df = df[df['PREDICTION'] == 2].reset_index(drop=True) # removes unecessary index column
    for i in tqdm(np.random.randint(0, len(spam_df), samples)):
        # generating 'n_samples' augmented texts
        for _ in range(repetitions):
            augmented_text = augmenter.augment(spam_df['DESCRIPTION'].iloc[i])
            augmented_texts.append(augmented_text)
    
    data = {
        'PREDICTION': 2,
        'DESCRIPTION': augmented_texts
    }
    aug_df = pd.DataFrame(data)
    df = shuffle(df.append(aug_df).reset_index(drop=True))
    return df

In [ ]:
aug_df1 = augmentMyData2(ll_dataset, augmenter, samples=200)

100%|██████████| 200/200 [02:58<00:00,  1.12it/s]


In [ ]:
def augmentMyData6(df, augmenter, repetitions=1, samples=200):
    augmented_texts = []
    # select only the minority class samples
    spam_df = df[df['PREDICTION'] == 6].reset_index(drop=True) # removes unecessary index column
    for i in tqdm(np.random.randint(0, len(spam_df), samples)):
        # generating 'n_samples' augmented texts
        for _ in range(repetitions):
            augmented_text = augmenter.augment(spam_df['DESCRIPTION'].iloc[i])
            augmented_texts.append(augmented_text)
    
    data = {
        'PREDICTION': 6,
        'DESCRIPTION': augmented_texts
    }
    aug_df = pd.DataFrame(data)
    df = shuffle(df.append(aug_df).reset_index(drop=True))
    return df

In [ ]:
aug_df2 = augmentMyData6(aug_df1, augmenter, samples=200)

100%|██████████| 200/200 [01:18<00:00,  2.54it/s]


In [ ]:
def augmentMyData9(df, augmenter, repetitions=1, samples=200):
    augmented_texts = []
    # select only the minority class samples
    spam_df = df[df['PREDICTION'] == 9].reset_index(drop=True) # removes unecessary index column
    for i in tqdm(np.random.randint(0, len(spam_df), samples)):
        # generating 'n_samples' augmented texts
        for _ in range(repetitions):
            augmented_text = augmenter.augment(spam_df['DESCRIPTION'].iloc[i])
            augmented_texts.append(augmented_text)
    
    data = {
        'PREDICTION': 9,
        'DESCRIPTION': augmented_texts
    }
    aug_df = pd.DataFrame(data)
    df = shuffle(df.append(aug_df).reset_index(drop=True))
    return df

In [ ]:
aug_df3 = augmentMyData9(aug_df2, augmenter, samples=200)

100%|██████████| 200/200 [02:40<00:00,  1.25it/s]


In [ ]:
def augmentMyData7(df, augmenter, repetitions=1, samples=200):
    augmented_texts = []
    # select only the minority class samples
    spam_df = df[df['PREDICTION'] == 7].reset_index(drop=True) # removes unecessary index column
    for i in tqdm(np.random.randint(0, len(spam_df), samples)):
        # generating 'n_samples' augmented texts
        for _ in range(repetitions):
            augmented_text = augmenter.augment(spam_df['DESCRIPTION'].iloc[i])
            augmented_texts.append(augmented_text)
    
    data = {
        'PREDICTION': 7,
        'DESCRIPTION': augmented_texts
    }
    aug_df = pd.DataFrame(data)
    df = shuffle(df.append(aug_df).reset_index(drop=True))
    return df

In [ ]:
aug_df4 = augmentMyData7(aug_df3, augmenter, samples=200)


100%|██████████| 200/200 [01:16<00:00,  2.62it/s]


In [ ]:
def augmentMyData8(df, augmenter, repetitions=1, samples=200):
    augmented_texts = []
    # select only the minority class samples
    spam_df = df[df['PREDICTION'] == 8].reset_index(drop=True) # removes unecessary index column
    for i in tqdm(np.random.randint(0, len(spam_df), samples)):
        # generating 'n_samples' augmented texts
        for _ in range(repetitions):
            augmented_text = augmenter.augment(spam_df['DESCRIPTION'].iloc[i])
            augmented_texts.append(augmented_text)
    
    data = {
        'PREDICTION': 8,
        'DESCRIPTION': augmented_texts
    }
    aug_df = pd.DataFrame(data)
    df = shuffle(df.append(aug_df).reset_index(drop=True))
    return df

In [ ]:
aug_df5 = augmentMyData8(aug_df4, augmenter, samples=200)

100%|██████████| 200/200 [02:23<00:00,  1.39it/s]


In [ ]:
def augmentMyData4(df, augmenter, repetitions=1, samples=200):
    augmented_texts = []
    # select only the minority class samples
    spam_df = df[df['PREDICTION'] == 4].reset_index(drop=True) # removes unecessary index column
    for i in tqdm(np.random.randint(0, len(spam_df), samples)):
        # generating 'n_samples' augmented texts
        for _ in range(repetitions):
            augmented_text = augmenter.augment(spam_df['DESCRIPTION'].iloc[i])
            augmented_texts.append(augmented_text)
    
    data = {
        'PREDICTION': 4,
        'DESCRIPTION': augmented_texts
    }
    aug_df = pd.DataFrame(data)
    df = shuffle(df.append(aug_df).reset_index(drop=True))
    return df

In [ ]:
aug_df6 = augmentMyData4(aug_df5, augmenter, samples=200)

100%|██████████| 200/200 [03:02<00:00,  1.09it/s]


In [ ]:
aug_df6['PREDICTION'].value_counts()

3    552
0    480
1    423
4    308
8    302
2    275
6    258
9    239
7    211
5    184
Name: PREDICTION, dtype: int64

##### 2. Split train-val-test

In [ ]:
# split train & test data (scikit learn helper function)
train, test = train_test_split(aug_df6,test_size=0.2, stratify=aug_df6[["PREDICTION"]])

In [ ]:
train["PREDICTION"].value_counts()

3    442
0    384
1    338
4    246
8    242
2    220
6    206
9    191
7    169
5    147
Name: PREDICTION, dtype: int64

In [ ]:
test["PREDICTION"].value_counts()

3    110
0     96
1     85
4     62
8     60
2     55
6     52
9     48
7     42
5     37
Name: PREDICTION, dtype: int64

In [ ]:
# the prediction column (train) is going to be turned into a 1-Hot vector
labels_df = pd.get_dummies(train["PREDICTION"])
labels_df.columns =["cat0","cat1","cat2","cat3","cat4","cat5","cat6","cat7","cat8","cat9"]

In [ ]:
train = pd.concat([train[["DESCRIPTION"]],labels_df],axis=1)

In [ ]:
# the prediction column (test) is going to be turned into a 1-Hot vector
labels_df1 = pd.get_dummies(test["PREDICTION"])
labels_df1.columns =["cat0","cat1","cat2","cat3","cat4","cat5","cat6","cat7","cat8","cat9"]

In [ ]:
test = pd.concat([test[["DESCRIPTION"]],labels_df1],axis=1)

In [ ]:
# train_data
train_data = list(train["DESCRIPTION"].values)
train_labels = list(train.drop("DESCRIPTION", axis=1).values)

In [ ]:
# val_data
val_data = list(test["DESCRIPTION"].values)
val_labels = list(test.drop("DESCRIPTION", axis=1).values)

##### 3. Encodings

In [ ]:
# name of the model of interest (distilbert + uncased)
# the poblem type is : multi_label_classification not like before : bianry classification

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, problem_type="multi_label_classification")

train_encodings = tokenizer(train_data, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_data, truncation=True, padding=True, max_length=128)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
# Dataset class where we create a Dataset 
# like BERT can accept it
class LLDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).to(dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
# Datasets of train, validation and testing
train_dataset = LLDataset(train_encodings, train_labels)
val_dataset = LLDataset(val_encodings, val_labels)

In [ ]:
train_dataset[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([  101, 12436,  2102,  3784,  2000,  1024,  3229,  2924,  1064,  7367,
          3593,  2226,  8945,  2721,  4478, 27617, 12672,  2368, 25682,  2692,
         22275, 12521,  2692, 24434,  2692,  2575, 17788,  2683,  1013,  2199,
          2692, 21926, 17465,  2692, 22275, 12521,  2692, 24434, 18613,  2692,
          2289,  2620, 21472, 17134,  2575,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0

##### 4. Model definition + training

In [ ]:
# Build the model by specifying the .cpkt, the number of labels

from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer)
num_labels= 10
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, 
                                                           num_labels=num_labels, 
                                                           problem_type="multi_label_classification")

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_steps=10,
    #push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2585
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1620


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.012900,0.062994,0.914596,0.950713,0.902628
2,0.017900,0.056088,0.928793,0.959815,0.921175
3,0.018300,0.059018,0.933230,0.961017,0.927357
4,0.009300,0.063247,0.931168,0.961446,0.925811
5,0.010200,0.060254,0.928571,0.958441,0.919629
6,0.010300,0.060566,0.933230,0.961017,0.925811
7,0.020000,0.062382,0.934057,0.961789,0.928903
8,0.017300,0.060607,0.931677,0.960158,0.927357
9,0.002400,0.061510,0.936434,0.963421,0.930448
10,0.008500,0.061260,0.933333,0.961704,0.924266


***** Running Evaluation *****
  Num examples = 647
  Batch size = 64
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-162
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-162/config.json
***** Running Evaluation *****
  Num examples = 647
  Batch size = 64
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-324
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-324/config.json
***** Running Evaluation *****
  Num examples = 647
  Batch size = 64
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-486
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-486/config.json
***** Running Evaluation *****
  Num examples = 647
  Batch size = 64
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-648
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-648/config.json
***** Running Evaluation *****
  Num examples = 647
  Batch size = 64
Saving model checkpoint to

TrainOutput(global_step=1620, training_loss=0.011238682926574975, metrics={'train_runtime': 209.0052, 'train_samples_per_second': 123.681, 'train_steps_per_second': 7.751, 'total_flos': 729101594595000.0, 'train_loss': 0.011238682926574975, 'epoch': 10.0})

In [ ]:
trainer.predict(val_dataset)



***** Running Prediction *****
  Num examples = 647
  Batch size = 64


PredictionOutput(predictions=array([[ -6.832945  ,  -6.920761  ,  -7.2778316 , ...,   5.9091225 ,
         -7.074945  ,  -6.854175  ],
       [  6.0597215 ,  -8.082486  ,  -8.343954  , ...,  -8.897014  ,
         -9.163775  ,  -7.2951097 ],
       [ -7.419408  ,   5.5627966 ,  -7.7271905 , ...,  -8.956051  ,
         -7.425079  ,  -7.762295  ],
       ...,
       [ -0.28636655,  -3.306943  , -10.655298  , ..., -12.008067  ,
        -10.374054  ,  -6.311328  ],
       [ -4.6439934 ,  -7.643302  ,  -8.742418  , ...,  -8.918697  ,
         -8.834777  ,  -7.864008  ],
       [  4.786857  ,  -5.3235273 ,  -8.969397  , ..., -10.109463  ,
         -7.7075267 ,  -8.364316  ]], dtype=float32), label_ids=array([[0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32), metrics={'test_loss': 0.061510395258665

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 647
  Batch size = 64


{'epoch': 10.0,
 'eval_accuracy': 0.9304482225656878,
 'eval_f1': 0.9364341085271317,
 'eval_loss': 0.061510395258665085,
 'eval_roc_auc': 0.9634209170530655,
 'eval_runtime': 1.0321,
 'eval_samples_per_second': 626.908,
 'eval_steps_per_second': 10.658}

##### 5. Inference

In [ ]:
# we can actually play with the data and create artificial
# new records
text = ["MOB/UTO/7374736517/000017/1 8 mm sent for ayin 000004210312153133254412791788"]
encoding = tokenizer(text, return_tensors="pt").to('cuda')

# forward pass
outputs = model(**encoding)
predictions = outputs.logits.argmax(-1)
int(predictions)

1

In [ ]:
text = ["TNF-ONABIYI ABIODUN OLUMIDEONABIYI ABIODUN OLUMIDE 000013210318081620000044927184"]
encoding = tokenizer(text, return_tensors="pt").to('cuda')

# forward pass
outputs = model(**encoding)
predictions = outputs.logits.argmax(-1)
int(predictions)


1

In [ ]:
import pandas as pd

In [ ]:
INPUT= '/content/drive/MyDrive/Bert_classification/input.csv'

In [ ]:

df = pd.read_csv(INPUT, sep = ';' )
df

,BANK_ID,DESCRIPTION
0,UBA,SMS/SMS CHARGES FEB 16TH 2021 - MAR 15TH 2021 ...
1,UBA,FGN STAMPDUTY FOR: 1 TRANSACTIONS ON 15-MAR- 21
2,UBA,FGN STAMPDUTY FOR: 1 TRANSACTIONS ON 18-MAR- 21
3,UBA,MOB/UTO/7445598102/00001\n3/Done 0000042103201...
4,UBA,TNF-MIDZAC LIMITED/IBR TRF IFO ERARIO BUSINESS...
5,UBA,MOB/UTO/7467889276/00000\n7/Done 0000042103222...
6,UBA,FGN STAMPDUTY FOR: 1 TRANSACTIONS ON 22-MAR- 21
7,UBA,TNF-RML 1729 LIMITED/IBR TRF IFO ERARIO BUSINE...
8,UBA,MOB/UTO/7553980900/00000\n7/Done 0000042103311...
9,UBA,10XXX..:Int. Pd.01-03-\n2021to31-03-2021 Inter...


In [ ]:
text = ["NIP Charge + VAT"]
encoding = tokenizer(text, return_tensors="pt").to('cuda')

# forward pass
outputs = model(**encoding)
predictions = outputs.logits.argmax(-1)
int(predictions)

3

In [ ]:
text = ["TRF FRM 420 PROPERTIES LIMITED TO ESEOSA OREOL"]
encoding = tokenizer(text, return_tensors="pt").to('cuda')

# forward pass
outputs = model(**encoding)
predictions = outputs.logits.argmax(-1)
int(predictions)

4

In [ ]:
text = ["LOAN INSURANCE PREMIUM/GLM-163555/CREDIT LIFE"]
encoding = tokenizer(text, return_tensors="pt").to('cuda')

# forward pass
outputs = model(**encoding)
predictions = outputs.logits.argmax(-1)
int(predictions)

2

In [ ]:
text = ["TNF-ONABIYI ABIODUN OLUMIDEONABIYI ABIODUN OLUMIDE 0000132103312159460000654"]
encoding = tokenizer(text, return_tensors="pt").to('cuda')

# forward pass
outputs = model(**encoding)
predictions = outputs.logits.argmax(-1)
int(predictions)

1

In [ ]:
text = ["VAT ON ACCT MAIN CHRG"]
encoding = tokenizer(text, return_tensors="pt").to('cuda')

# forward pass
outputs = model(**encoding)
predictions = outputs.logits.argmax(-1)
int(predictions)

3

In [ ]:
# Function that predict with model from the notebook

def classifier(INPUT_FILE): # not possible when using lambda function 
   
  df = pd.read_csv(INPUT_FILE, sep=';')
  transaction_list = [] 
  pred_list= []
  for index, row in df.iterrows():
    text = row['DESCRIPTION']
    encoding = tokenizer(text, return_tensors="pt").to('cuda')# no cuda with lambda 
    outputs = model(**encoding)
    predictions = outputs.logits.argmax(-1)
    int(predictions)
    transaction_list.append(text)
    pred_list.append(int(predictions))
     
    print(row['BANK_ID'],  '|' , 'The transcation '+ '"' + (row['DESCRIPTION']) + 
        '"', 'corresponds to the category' , int(predictions))
    
  df1 = pd.DataFrame(list(zip(transaction_list, pred_list)), columns = ['TRANSACTION', 'CATEGORY'])

  
  df1.to_csv('/content/drive/MyDrive/Bert_classification/output_file.csv')
  # we can reverse the one hot encoding with  idxmax


In [ ]:
# script for predict
import torch
import csv

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, BertConfig
from transformers import AdamW
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import DistilBertTokenizerFast, AutoTokenizer
from sklearn.model_selection import train_test_split
import pandas as pd

# we have to add the loaded_model import here or another script
def classifier(INPUT_FILE, loaded_model): # not possible when using lambda function 
   
  df = pd.read_csv(INPUT_FILE, sep =';' )
  transaction_list = [] 
  pred_list= []
  pred_code_list=[]
  for index, row in df.iterrows():
    text = row['DESCRIPTION']
    model_ckpt = "distilbert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_ckpt, problem_type="multi_label_classification")
    encoding = tokenizer(text, return_tensors="pt")#
    outputs = loaded_model(**encoding)
    predictions = outputs.logits.argmax(-1)
    int(predictions)
    transaction_list.append(text)
    pred_code_list.append(int(predictions))
    if int(predictions) == 0:
      pred_list.append('TRANSFER')
    elif int(predictions)== 1:
      pred_list.append('PURCHASE')
    elif int(predictions) == 2:
      pred_list.append('LOAN')
    elif int(predictions)== 3:
      pred_list.append('CHARGES')
    elif int(predictions)== 4:
      pred_list.append('SALARY')
    elif int(predictions) == 5:
      pred_list.append('CASH')
    elif int(predictions)== 6:
      pred_list.append('REVERSAL')
    elif int(predictions)== 7:
      pred_list.append('CHEQUE')
    elif int(predictions)== 8:
      pred_list.append('PAYMENT')
    elif int(predictions)== 9:
      pred_list.append('UNKNOWN')



     
    print(row['BANK_ID'],  '|' , 'The transcation '+ '"' + (row['DESCRIPTION']) + 
        '"', 'corresponds to the category ' , int(predictions))
    
  df1 = pd.DataFrame(list(zip(transaction_list, pred_list, pred_code_list)), columns = ['TRANSACTION', 'CATEGORY', 'CATEGORY_CODE'])

  df1.to_csv('my_ouptput.csv')

In [ ]:
INPUT_FILE= '/content/drive/MyDrive/Bert_classification/input.csv'

In [ ]:
classifier(INPUT_FILE)

UBA | The transcation "SMS/SMS CHARGES FEB 16TH 2021 - MAR 15TH 2021 Fee Col / ECHARGESUB" corresponds to the category 3
UBA | The transcation "FGN STAMPDUTY FOR: 1 TRANSACTIONS ON 15-MAR- 21" corresponds to the category 3
UBA | The transcation "FGN STAMPDUTY FOR: 1 TRANSACTIONS ON 18-MAR- 21" corresponds to the category 3
UBA | The transcation "MOB/UTO/7445598102/00001
3/Done 0000042103201020065514643
93322" corresponds to the category 1
UBA | The transcation "TNF-MIDZAC LIMITED/IBR TRF IFO ERARIO BUSINESS ENT 0000122103221305290002333
71063" corresponds to the category 0
UBA | The transcation "MOB/UTO/7467889276/00000
7/Done 0000042103222009154400234
15139" corresponds to the category 1
UBA | The transcation "FGN STAMPDUTY FOR: 1 TRANSACTIONS ON 22-MAR- 21" corresponds to the category 3
UBA | The transcation "TNF-RML 1729 LIMITED/IBR TRF IFO ERARIO BUSINESS E 0000122103311350110002367
50430" corresponds to the category 0
UBA | The transcation "MOB/UTO/7553980900/00000
7/Done 00000421

In [ ]:
% cd /content/drive/MyDrive/Bert_classification

/content/drive/MyDrive/Bert_classification


In [ ]:
! pwd

/content/drive/MyDrive/Bert_classification


In [ ]:
# save the trained model
torch.save(model,'/content/drive/MyDrive/Bert_classification/trained_model.pt')

In [ ]:
# load the saved model
trained_model = torch.load('/content/drive/MyDrive/Bert_classification/trained_model.pt')

In [ ]:
# use the trained model to predict 

In [ ]:
text = ["carte iop shop"]
encoding = tokenizer(text, return_tensors="pt").to('cuda')

# forward pass
outputs = trained_model(**encoding)
predictions = outputs.logits.argmax(-1)
int(predictions)

1

In [ ]:
!python main.py 

UBA | The transcation "SMS/SMS CHARGES FEB 16TH 2021 - MAR 15TH 2021 Fee Col / ECHARGESUB" is a transaction from category  3
UBA | The transcation "FGN STAMPDUTY FOR: 1 TRANSACTIONS ON 15-MAR- 21" is a transaction from category  3
UBA | The transcation "FGN STAMPDUTY FOR: 1 TRANSACTIONS ON 18-MAR- 21" is a transaction from category  3
UBA | The transcation "MOB/UTO/7445598102/00001
3/Done 0000042103201020065514643
93322" is a transaction from category  1
UBA | The transcation "TNF-MIDZAC LIMITED/IBR TRF IFO ERARIO BUSINESS ENT 0000122103221305290002333
71063" is a transaction from category  0
UBA | The transcation "MOB/UTO/7467889276/00000
7/Done 0000042103222009154400234
15139" is a transaction from category  1
UBA | The transcation "FGN STAMPDUTY FOR: 1 TRANSACTIONS ON 22-MAR- 21" is a transaction from category  3
UBA | The transcation "TNF-RML 1729 LIMITED/IBR TRF IFO ERARIO BUSINESS E 0000122103311350110002367
50430" is a transaction from category  0
UBA | The transcation "MOB/UTO/

In [ ]:
%cd /content/drive/MyDrive/results

/content/drive/MyDrive/results


In [ ]:
!python main.py 

UBA | The transcation "SMS/SMS CHARGES FEB 16TH 2021 - MAR 15TH 2021 Fee Col / ECHARGESUB" is a transaction from category  3
UBA | The transcation "FGN STAMPDUTY FOR: 1 TRANSACTIONS ON 15-MAR- 21" is a transaction from category  3
UBA | The transcation "FGN STAMPDUTY FOR: 1 TRANSACTIONS ON 18-MAR- 21" is a transaction from category  3
UBA | The transcation "MOB/UTO/7445598102/00001
3/Done 0000042103201020065514643
93322" is a transaction from category  1
UBA | The transcation "TNF-MIDZAC LIMITED/IBR TRF IFO ERARIO BUSINESS ENT 0000122103221305290002333
71063" is a transaction from category  0
UBA | The transcation "MOB/UTO/7467889276/00000
7/Done 0000042103222009154400234
15139" is a transaction from category  1
UBA | The transcation "FGN STAMPDUTY FOR: 1 TRANSACTIONS ON 22-MAR- 21" is a transaction from category  3
UBA | The transcation "TNF-RML 1729 LIMITED/IBR TRF IFO ERARIO BUSINESS E 0000122103311350110002367
50430" is a transaction from category  0
UBA | The transcation "MOB/UTO/